## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## And so on and so forth...

In [3]:
img = cv2.imread('../camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)



## Color And Gradient Thresholding 

In [4]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))

    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)    

    binary_output[(scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1])] = 1
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

def hsv_select(img, thresh=(0, 255)):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    binary_output = np.zeros_like(v_channel)
    binary_output[(v_channel > thresh[0]) & (v_channel <= thresh[1])] = 1
    return binary_output

def color_threshold(image, sthresh, vthresh):
    s_binary = hls_select(image, sthresh)
    v_binary = hsv_select(image, vthresh)
    
    output = np.zeros_like(s_binary)
    #any overlap results in a pixel being drawn
    output[(s_binary == 1) & (v_binary == 1) == 1] = 1
    return output

In [5]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        self.n = 5

        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.all_x = []  
        #y values for detected line pixels
        self.all_y = []
        
        self.all_a = []
        self.all_b = []
        self.all_c = []
        
        
    def bestx(self):
        return np.mean(self.recent_xfitted)
    
    def add_coefficients(self, coefficients):
        self.all_a.append(coefficients[0])
        self.all_b.append(coefficients[1])
        self.all_c.append(coefficients[2])
        
    def current_fit(self):
        return [self.all_a[-1], self.all_b[-1], self.all_c[-1]]
    
    def average_coefficients(self, farback=0):
        return [np.mean(self.all_a[-farback:]), np.mean(self.all_b[-farback:]), np.mean(self.all_c[-farback:])]
    
    def add_x(self, x):
        self.all_x.append(x)

        
    def best_x(self):
        return np.mean(self.all_x[-1 * self.n:])
    
    def add_y(self, y):
        self.all_y.append(y)
        
    def currentx(self):
        return self.all_x[-1]

    def currenty(self):
        return self.all_y[-1]
    
    def best_y(self):
        return np.mean(self.all_y[-1 * self.n])
    
    def best_coefficients(self):
        self.average_coefficients(self.n)
    
    

        

In [17]:
fname = '../test_images/test1.jpg'
img = cv2.imread(fname)
img = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('./undistorted.jpg', img)

True

In [6]:
test_images = glob.glob('../test_images/*.jpg')

class Tracker:
    def __init__(self):
        self.previousLeftRadius = None
        self.previousRightRaidus = None
        self.previousWindow = None
        self.previousLeftRadius = None
        self.previousRightRadius = None
        self.previousRightFitx = None
        self.previousLeftFitx = None
        self.previousCenterDiff = []
        
        self.previousLeftXBase = None
        self.previousRightXBase = None
        self.left = Line()
        self.right = Line()
        
    def histogramize(self, binary_warped):
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 
        
        self.previousLeftXBase = leftx_current
        self.previousRightXBase = rightx_current

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
        
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        #Draw the lane onto the warped blank image
#         cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
#         cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
#         result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
#         cv2.imwrite('./lanefinding.jpg', result)
#         plt.plot(left_fitx, ploty, color='yellow')
#         plt.plot(right_fitx, ploty, color='yellow')
#         plt.xlim(0, 1280)
#         plt.ylim(720, 0)

        
        return (left_fitx, right_fitx, ploty)
        

    def image_width(self, image):
        return image.shape[1]

    def image_height(self, image):
        return image.shape[0]

    def src_points(self, image):
        width = self.image_width(image)
        height = self.image_height(image)
        bottom_width = 0.76
        middle_width = 0.08
        height_pct = 0.62
        bottom_trim = 0.935

        top_left = [width * (0.5 - middle_width/2), (height * height_pct)]
        top_right = [width * (0.5 + middle_width/2),(height * height_pct)]
        bottom_right = [(width *(0.5 + bottom_width/2)), (height * bottom_trim)]
        bottom_left = [(width * (0.5 - bottom_width/2)), (height * bottom_trim)]

        return np.float32([ top_left, top_right, bottom_right, bottom_left])

    def process_image(self, img):
        img = cv2.undistort(img, mtx, dist, None, mtx)

        #Make a gradient/directional mask
        preprocess_image = np.zeros_like(img[:,:,0])
        gradx = abs_sobel_thresh(img, orient='x', thresh=(15,255))
        grady = abs_sobel_thresh(img, orient='y', thresh=(30,255))

        c_binary = color_threshold(img, sthresh=(120,255), vthresh=(60,255))
        preprocess_image[(((gradx == 1) & (grady ==1)) | (c_binary == 1))] = 255
#         cv2.imwrite('./thresholding.jpg', preprocess_image)

        width = self.image_width(preprocess_image)
        height = self.image_height(preprocess_image)

        source = self.src_points(preprocess_image)

        #A quarter of the width of the image
        offset = width * 0.2

        #top left, top right, bottom right, bottom left
        destination = np.float32([[offset, 0], [width-offset, 0], [width-offset, height], [offset, height]])

        #Perspective Transform
        M = cv2.getPerspectiveTransform(source, destination)
        Minv = cv2.getPerspectiveTransform(destination, source)
        warped = cv2.warpPerspective(preprocess_image, M, (width, height), flags=cv2.INTER_LINEAR)
        
        left_fitx, right_fitx, ploty = self.histogramize(warped)
        
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        
        #in between the two lane lines
        camera_center = (left_fitx[-1] + right_fitx[-1])/2 
        
        #Calculate the car's distance from center
        center_diff = (camera_center - warped.shape[1]/2) * xm_per_pix
        side_pos = 'left'
        if center_diff <= 0:
            side_pos = 'right'
        y_eval = np.max(ploty)
        
        # Calculate the new radii of curvature
        left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
        
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

        #Anomaly detection, if the car's distance from center exceeds the mean by more than .315 meters, then we use 
        #the previous line
        diff = "lol"
        if (self.previousLeftRadius and self.previousRightRadius and self.previousCenterDiff):
            diff = abs(abs(center_diff) - np.mean(self.previousCenterDiff[-20:]))
            if diff > 0.315:
                ploty = self.previousPloty
                left_curverad = self.previousLeftRadius
                right_curverad = self.previousRightRadius
                left_fitx = self.previousLeftFitx
                right_fitx = self.previousRightFitx
                center_diff = self.previousCenterDiff[-1]
            else:
                self.previousPloty = ploty
                self.previousLeftRadius = left_curverad
                self.previousRightRadius = right_curverad
                self.previousLeftFitx = left_fitx
                self.previousRightFitx = right_fitx
                self.previousCenterDiff.append(abs(center_diff))
        else:
            self.previousPloty = ploty
            self.previousLeftRadius = left_curverad
            self.previousRightRadius = right_curverad
            self.previousLeftFitx = left_fitx
            self.previousRightFitx = right_fitx
            self.previousCenterDiff.append(abs(center_diff))
        

        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(img, 1, newwarp, 0.3, 0)
        cv2.putText(result, 'Left radius: ' + str(round(left_curverad, 3)) + 'm', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(result, 'Right radius: '+ str(round(right_curverad, 3)) + 'm', (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        cv2.putText(result, 'Car is '+ str(round(center_diff,2)) + 'm ' + str(side_pos) + 'of center', (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        cv2.putText(result, 'Diff is '+ str(diff), (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
        
        
        return result



In [7]:
for idx, fname in enumerate(test_images):
    tracker = Tracker()
    img = cv2.imread(fname)
    result = tracker.process_image(img)
#     write_name = 'output' + str(idx) + '.jpg'
#     plt.figure()
#     plt.imshow(result)
#     cv2.imwrite(write_name, result)

In [7]:
from moviepy.editor import VideoFileClip

output = './output.mp4'
tracker = Tracker()
clip1 = VideoFileClip("../project_video.mp4")
clip = clip1.fl_image(tracker.process_image) #NOTE: this function expects color images!!
%time clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ./output.mp4
[MoviePy] Writing video ./output.mp4


100%|█████████▉| 1260/1261 [02:47<00:00,  7.28it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output.mp4 

CPU times: user 3min 51s, sys: 39.4 s, total: 4min 30s
Wall time: 2min 48s
